# Brazilian E-Commerce Analysis (Olist)
## SQL & Python Project

This project analyzes data from over 100k orders in Brazilian e-commerce between 2016 and 2018. 
The main goal is to answer business questions regarding logistics performance, payment behaviors, and sales trends.

**Tech Stack:**
* **SQL (SQLite)** - for data manipulation and joining tables
* **Python (Pandas)** - for data loading and processing
* **Seaborn/Matplotlib** - for data visualization

In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Visualization configuration
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

### 1. Environment Setup & Data Loading
We load CSV files and create a temporary in-memory SQL database.

In [ ]:
# File list (ensure these files are in the root directory)
files = {
    'orders': 'olist_orders_dataset.csv',
    'items': 'olist_order_items_dataset.csv',
    'products': 'olist_products_dataset.csv',
    'customers': 'olist_customers_dataset.csv',
    'reviews': 'olist_order_reviews_dataset.csv',
    'payments': 'olist_order_payments_dataset.csv',
    'category_trans': 'product_category_name_translation.csv'
}

conn = sqlite3.connect(':memory:')

print("Loading data into SQL database...")
for name, file_name in files.items():
    try:
        pd.read_csv(file_name).to_sql(name, conn, index=False, if_exists='replace')
        print(f"✓ Table {name} loaded")
    except FileNotFoundError:
        print(f"❌ Missing file: {file_name}")

### 2. Logistics Analysis: Delivery Time by State
**Business Question:** Which regions experience the longest delivery times?

We join `orders`, `customers`, and `order_items` tables to calculate the average delivery time (difference between purchase timestamp and delivered customer date).

In [ ]:
query_logistics = """
SELECT
    c.customer_state AS State,
    COUNT(DISTINCT o.order_id) AS Total_Orders,
    ROUND(AVG(julianday(o.order_delivered_customer_date) - julianday(o.order_purchase_timestamp)), 1) AS Avg_Delivery_Time_Days
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
WHERE o.order_status = 'delivered'
GROUP BY c.customer_state
ORDER BY Avg_Delivery_Time_Days DESC;
"""

df_logistics = pd.read_sql(query_logistics, conn)

# Visualization
plt.figure(figsize=(14, 6))
sns.barplot(data=df_logistics, x='State', y='Avg_Delivery_Time_Days', hue='State', palette='viridis', legend=False)
plt.title('Average Delivery Time by State (Days)')
plt.xlabel('State Code')
plt.ylabel('Days')
plt.axhline(y=df_logistics['Avg_Delivery_Time_Days'].mean(), color='r', linestyle='--', label='National Average')
plt.legend()
plt.show()

**Key Insights:**
* Northern states (RR, AP, AM) suffer from drastically longer delivery times (>25 days) compared to the South (SP, RJ ~8-10 days).
* This indicates a need for supply chain optimization in the Amazon region or setting better customer expectations for those states.

### 3. Payment Analysis: Installments vs. Order Value
**Business Question:** Do customers spend more when they split payments into installments?

In [ ]:
query_installments = """
SELECT
    payment_installments AS Installments,
    COUNT(DISTINCT order_id) AS Total_Transactions,
    ROUND(AVG(payment_value), 2) AS Avg_Order_Value
FROM order_payments
WHERE payment_type = 'credit_card' AND payment_installments > 0
GROUP BY payment_installments
ORDER BY payment_installments ASC;
"""

df_installments = pd.read_sql(query_installments, conn)

# Visualization
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_installments, x='Installments', y='Avg_Order_Value', marker='o', color='green')
plt.title('Correlation: Number of Installments vs. Average Order Value')
plt.ylabel('Avg Order Value (BRL)')
plt.xlabel('Number of Installments')
plt.grid(True)
plt.show()

**Key Insights:**
* There is a strong positive correlation between installments and order value.
* Customers paying in 1 installment spend an average of ~96 BRL, whereas those choosing 10 installments spend over 400 BRL.
* **Recommendation:** Promote installment plans for high-ticket items to increase conversion rates.